# Import libraries

In [3]:
import torch
from piq import ssim, multi_scale_ssim as ms_ssim
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot

from src.metrics import *
from src.utils import *
from src.datasets import ExcavatorDataset
from src.config import TRANSFORMER, ROOT, DEVICE
from models.Segmentation import DeepLabV3Model, DeepLabV3PlusModel, PyramidAttentionNetworkModel, UNetModel

False


# Load dataset

In [3]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', plot=False)
print("Number of train imgs: ", len(train_dataset))
test_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', plot=False)
print("Number of val imgs: ", len(test_dataset))
cls_colors = train_dataset.class_colors

Number of train imgs:  1782
Number of val imgs:  187


/home/ais/Bachelorarbeit/similarity_metrics_of_images/src/datasets.py:308: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


# Test 1. VLAD (PASS)

In [3]:
for i in range(30):
    num_cls= np.random.choice([32, 64, 128, 256])
    feature = np.random.choice(['sift', 'root_sift'])
    dataset_choice = np.random.choice(['train', 'validation'])
    if dataset_choice == 'train':
        dataset = train_dataset
    else:
        dataset = test_dataset
    print("Chosen dataset:", dataset_choice)
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(dataset)-1)
        idx_2 = random.randint(0, len(dataset)-1)

    image_1, *_ ,path1= dataset[idx_1]
    image_2, *_, path2 = dataset[idx_2]
    k_means = load_model(f'{ROOT}/models/pickle_model_files/k_means_model_k{num_cls}_{feature}.pkl')

    vlad_1 = VLAD(image_1, k_means=k_means, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(vlad_1))
    vlad_2 = VLAD(image_2, k_means=k_means, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(vlad_2))

    print("VLAD cosine similarity:", res:=cosine_similarity(vlad_1.reshape(1, -1), vlad_2.reshape(1, -1))[0][0])

    hdf_file = load_hdf5(f'{ROOT}/res/vlad/{dataset_choice}/k_means_model_k{num_cls}_{feature}.h5')

    vlad_check_1 = hdf_file[os.path.basename(path1)]
    vlad_check_2 = hdf_file[os.path.basename(path2)]
    print("Check VLAD cosine similarity:", res_check:=cosine_similarity(vlad_check_1.reshape(1, -1), vlad_check_2.reshape(1, -1))[0][0])
    if not abs(res - res_check) < 1e-5:
        raise ValueError("VLAD cosine similarity is not equal to the check value")

print("Test passed")

Chosen dataset: train
Length of VLAD vector: 16384
Length of VLAD vector: 16384
VLAD cosine similarity: 0.02222677560055967
Check VLAD cosine similarity: 0.02222677560055967
Chosen dataset: train
Length of VLAD vector: 32768
Length of VLAD vector: 32768
VLAD cosine similarity: 0.025105189288985058
Check VLAD cosine similarity: 0.025105189288985058
Chosen dataset: validation
Length of VLAD vector: 4096
Length of VLAD vector: 4096
VLAD cosine similarity: -0.15603733185873025
Check VLAD cosine similarity: -0.15603733185873025
Chosen dataset: validation
Length of VLAD vector: 4096
Length of VLAD vector: 4096
VLAD cosine similarity: 0.06508906291638653
Check VLAD cosine similarity: 0.06508906291638653
Chosen dataset: train
Length of VLAD vector: 16384
Length of VLAD vector: 16384
VLAD cosine similarity: 0.0012902705248705468
Check VLAD cosine similarity: 0.0012902705248705468
Chosen dataset: validation
Length of VLAD vector: 32768
Length of VLAD vector: 32768
VLAD cosine similarity: 0.01370

# 2. Fisher Vector (PASS)

In [4]:
for i in range(30):
    num_cls= np.random.choice([32, 64, 128, 256])
    feature = np.random.choice(['sift', 'root_sift'])
    dataset_choice = np.random.choice(['train', 'validation'])
    if dataset_choice == 'train':
        dataset = train_dataset
    else:
        dataset = test_dataset
    print("Chosen dataset:", dataset_choice)
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(dataset)-1)
        idx_2 = random.randint(0, len(dataset)-1)

    image_1, *_ ,path1= dataset[idx_1]
    image_2, *_, path2 = dataset[idx_2]
    gmm = load_model(f'{ROOT}/models/pickle_model_files/gmm_model_k{num_cls}_{feature}.pkl')

    fisher_1 = FisherVector(image_1, gmm=gmm, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(fisher_1))
    fisher_2 = FisherVector(image_2, gmm=gmm, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(fisher_2))

    print("Fisher cosine similarity:", res:=cosine_similarity(fisher_1.reshape(1, -1), fisher_2.reshape(1, -1))[0][0])

    hdf_file = load_hdf5(f'{ROOT}/res/fisher/{dataset_choice}/gmm_model_k{num_cls}_{feature}.h5')

    fisher_check_1 = hdf_file[os.path.basename(path1)]
    fisher_check_2 = hdf_file[os.path.basename(path2)]
    print("Check Fisher cosine similarity:", res_check:=cosine_similarity(fisher_check_1.reshape(1, -1), fisher_check_2.reshape(1, -1))[0][0])
    if not abs(res - res_check) < 1e-5:
        raise ValueError("Fisher cosine similarity is not equal to the check value")

print("Test passed")


Chosen dataset: train
Length of VLAD vector: 32896
Length of VLAD vector: 32896
Fisher cosine similarity: 0.07895228838656596
Check Fisher cosine similarity: 0.07895228838656596
Chosen dataset: validation
Length of VLAD vector: 65792
Length of VLAD vector: 65792
Fisher cosine similarity: 0.05751707274923919
Check Fisher cosine similarity: 0.05751707274923919
Chosen dataset: train
Length of VLAD vector: 32896
Length of VLAD vector: 32896
Fisher cosine similarity: -0.013070390459632975
Check Fisher cosine similarity: -0.013070390459632975
Chosen dataset: train
Length of VLAD vector: 65792
Length of VLAD vector: 65792
Fisher cosine similarity: 0.021015846129207903
Check Fisher cosine similarity: 0.021015846129207903
Chosen dataset: train
Length of VLAD vector: 65792
Length of VLAD vector: 65792
Fisher cosine similarity: -0.042273249011004904
Check Fisher cosine similarity: -0.042273249011004904
Chosen dataset: validation
Length of VLAD vector: 16448
Length of VLAD vector: 16448
Fisher cos

# 3. SSIM, train vs train (PASS)

In [ ]:
for i in range(30):
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(train_dataset)-1)
        idx_2 = random.randint(0, len(train_dataset)-1)

    sigma = np.random.choice([0, 2, 4, 6, 8, 10])
    print(f"Chosen sigma for i={i}: ", sigma)
    image_1, *_ ,path1= train_dataset[idx_1]
    print(path1)
    image_2, *_, path2 = train_dataset[idx_2]

    image_1 = TRANSFORMER(image_1)
    image_2 = TRANSFORMER(image_2)

    if sigma != np.array(0):
        image_1 = gaussian_blur(image_1, sigma=sigma, kernel_size = 2*int(3*sigma)+1)
        image_2 = gaussian_blur(image_2, sigma=sigma, kernel_size = 2*int(3*sigma)+1)

    ssim = float(SSIM(image_1, image_2).value.item())
    ms_ssim = float(MS_SSIM(image_1, image_2).value.item())

    hdf_file = load_hdf5(f'{ROOT}/res/ssim/within_train/ssim_sigma{sigma}_sorted.h5')

    path_1_check = None
    idx_arr_1 = None
    path_2_check = None
    idx_arr_2 = None

    for i, path in enumerate(hdf_file['image_paths']):
        if isinstance(path, bytes):
            path = path.decode('utf-8')
        if os.path.basename(path1) == os.path.basename(path):
            idx_arr_1 = i
        if os.path.basename(path2) == os.path.basename(path):
            idx_arr_2 = i

    ssim_check = hdf_file['ssim'][idx_arr_2]
    ms_ssim_check = hdf_file['ms_ssim'][idx_arr_2]
    print("Check SSIM:", ssim_res_check:=ssim_check[idx_arr_1])
    print("Check MS-SSIM:", ms_ssim_res_check:=ms_ssim_check[idx_arr_1])
    if not abs(ssim - ssim_res_check) < 1e-5:
       raise ValueError("SSIM is not equal to the check value")
    if not abs(ms_ssim - ms_ssim_res_check) < 1e-5:
         raise ValueError("MS-SSIM is not equal to the check value")

print("Test passed")

Chosen sigma for i=0:  0
/home/ais/Bachelorarbeit/similarity_metrics_of_images/excavator_dataset_w_masks/train/roller/0040284_jpg.rf.dd54b8924da441588e73fbd80d399bb3.jpg
Check SSIM: 0.07677575
Check MS-SSIM: 0.029960733
Chosen sigma for i=1:  8
/home/ais/Bachelorarbeit/similarity_metrics_of_images/excavator_dataset_w_masks/train/caterpillar/002819_jpg.rf.260fcb3d76ccfde90551a3d322442564.jpg
Check SSIM: 0.60539484
Check MS-SSIM: 0.3891294
Chosen sigma for i=2:  4
/home/ais/Bachelorarbeit/similarity_metrics_of_images/excavator_dataset_w_masks/train/caterpillar/002650_jpg.rf.225eaea3900c3d06acd34557054cb522.jpg


## 4. SSIM, train vs val

In [9]:
# TODO
torch.__version__

'2.5.1+cu124'

# 5. Model predictions, IoU, all

In [7]:
# DeepLabV3
dlv3 =DeepLabV3Model(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3_HybridFocalDiceLoss.pt')

# DeepLabV3Plus
dlv3p = DeepLabV3PlusModel(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3Plus_HybridFocalDiceLoss.pt')

# UNet
unet = UNetModel(model_path=f'{ROOT}/models/torch_model_files/UNet_HybridFocalDiceLoss.pt')

# Pyramid Attention Network
pan = PyramidAttentionNetworkModel(model_path=f'{ROOT}/models/torch_model_files/PyramidAttentionNetwork_HybridFocalDiceLoss.pt')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
def plot_image_and_mask(image: torch.Tensor, gt_mask: torch.Tensor, pred_mask: torch.Tensor=None):
    """
    Plot image, ground truth mask and predicted mask (if available).

    :param image:
    :param gt_mask:
    :param pred_mask:
    :return:
    """
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 3, 1)
    if image.shape[0] == 3:
        image = image.permute(1, 2, 0)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Image')
    plt.subplot(1, 3, 2)
    if gt_mask.shape[0] == 3:
        gt_mask = gt_mask.permute(1, 2, 0)
    plt.imshow(gt_mask)
    plt.axis('off')
    plt.title('Ground Truth Mask')

    if pred_mask is None:
        plt.show()
        return

    plt.subplot(1, 3, 3)
    if pred_mask.shape[0] == 3:
        pred_mask = pred_mask.permute(1, 2, 0)
    plt.imshow(pred_mask)
    plt.axis('off')
    plt.title('Predicted Mask')
    plt.show()

for i in range(2):
    idx = random.randint(0, len(train_dataset)-1)
    model = np.random.choice([dlv3, dlv3p, unet, pan])
    print("Chosen model:", model.model.__class__.__name__)

    dataset_choice = np.random.choice(['train', 'val'])
    if dataset_choice == 'train':
        dataset = train_dataset
    else:
        dataset = test_dataset

    image, mask, path = dataset[idx]
    path_basename = os.path.basename(path)

    image = TRANSFORMER(image)

    mask = TRANSFORMER(mask)
    mask = rgb_to_mask(mask, cls_colors)


    _, pred_mask = model.predict_single_image(image, mask, raw_output=False)

    _, raw_pred= model.predict_single_image(image, mask, raw_output=True)
    mask_one_hot = one_hot(mask, num_classes=12).permute(2, 0, 1)
    iou_manual = multiclass_iou(mask_one_hot, raw_pred)

    model_paths = {
        dlv3: f'{ROOT}/res/model_performance/{dataset_choice}_iou_dlv3.h5',
        dlv3p: f'{ROOT}/res/model_performance/{dataset_choice}_iou_dlv3p.h5',
        unet: f'{ROOT}/res/model_performance/{dataset_choice}_iou_unet.h5',
        pan: f'{ROOT}/res/model_performance/{dataset_choice}_iou_pan.h5'
    }

    iou_data = load_hdf5(model_paths[model])
    target_iou = iou_data[path_basename]

    if not abs(iou_manual - target_iou) < 1e-5:
        raise ValueError("IoU difference is not equal to the check value")

    plot_image_and_mask(image, mask, pred_mask)
print("All tests passed")

NameError: name 'dlv3' is not defined

# 6. Model predictions, IoU, only classes contained in train mask

In [1]:
data = load_hdf5(f'{ROOT}/res/confidence_vectors/dlv3_only_classes_in_train.h5')

NameError: name 'load_hdf5' is not defined